# Ejercicio 12
Obtenga la matriz de distancias euclídeas para todos los contenidos que están en Marte. ¿Cuáles son los dos contenidos que están a menor distancia?

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
df_geotags = sqlContext.read.parquet('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/geo_tags.parquet', header=True, inferSchema=True)

In [ ]:
df_geotags

DataFrame[gt_id: bigint, gt_page_id: bigint, gt_globe: string, gt_primary: bigint, gt_lat: double, gt_lon: double, gt_dim: bigint, gt_type: string, gt_name: string, gt_country: string, gt_region: string]

In [ ]:
rdd_geotags = df_geotags.rdd

In [ ]:
rdd_geotags.take(5)

[Row(gt_id=158041, gt_page_id=4328020, gt_globe='earth', gt_primary=0, gt_lat=41.50749969, gt_lon=-5.81750011, gt_dim=1000, gt_type=None, gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844377, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.73799992, gt_lon=-89.29199982, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844381, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.70199966, gt_lon=-89.20800018, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844383, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.39999962, gt_lon=-89.75, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844388, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=12.85000038, gt_lon=-88.0, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None)]

In [ ]:
rdd_geotags.filter(lambda x: x.gt_globe == 'mars').take(10)

[Row(gt_id=74461565, gt_page_id=7473734, gt_globe='mars', gt_primary=0, gt_lat=16.9, gt_lon=141.7, gt_dim=1000, gt_type=None, gt_name='Corinto', gt_country=None, gt_region=None),
 Row(gt_id=74461566, gt_page_id=7473734, gt_globe='mars', gt_primary=0, gt_lat=37.4, gt_lon=126.4, gt_dim=1000, gt_type=None, gt_name='Dilly', gt_country=None, gt_region=None),
 Row(gt_id=74461567, gt_page_id=7473734, gt_globe='mars', gt_primary=0, gt_lat=-17.7, gt_lon=199.9, gt_dim=1000, gt_type=None, gt_name='Gratteri', gt_country=None, gt_region=None),
 Row(gt_id=74461568, gt_page_id=7473734, gt_globe='mars', gt_primary=0, gt_lat=16.3, gt_lon=125.9, gt_dim=1000, gt_type=None, gt_name='Tomini', gt_country=None, gt_region=None),
 Row(gt_id=74461569, gt_page_id=7473734, gt_globe='mars', gt_primary=0, gt_lat=-28.7, gt_lon=226.9, gt_dim=1000, gt_type=None, gt_name='Zumba', gt_country=None, gt_region=None),
 Row(gt_id=74461570, gt_page_id=7473734, gt_globe='mars', gt_primary=0, gt_lat=7.7, gt_lon=166.0, gt_dim=10

In [ ]:
marsLocationById = rdd_geotags.filter(lambda x: x.gt_globe == 'mars').map(lambda x: (x.gt_page_id,(x.gt_lat,x.gt_lon,1)))\
            .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1],x[2]+y[2])).map(lambda x: (x[0],(x[1][0]/x[1][2],x[1][1]/x[1][2])))
marsLocationById.take(5)

[(7473734, (7.7714285714285705, 162.6)),
 (2963878, (-2.2688000000000015, 161.07280000000003)),
 (5773024, (48.269, 134.01)),
 (222604, (0.0, 260.0)),
 (559924, (48.269, 134.01))]

In [ ]:
marsLocationById.cartesian(marsLocationById).take(10)

[((7473734, (7.7714285714285705, 162.6)),
  (7473734, (7.7714285714285705, 162.6))),
 ((7473734, (7.7714285714285705, 162.6)),
  (2963878, (-2.2688000000000015, 161.07280000000003))),
 ((7473734, (7.7714285714285705, 162.6)), (5773024, (48.269, 134.01))),
 ((7473734, (7.7714285714285705, 162.6)), (222604, (0.0, 260.0))),
 ((7473734, (7.7714285714285705, 162.6)), (559924, (48.269, 134.01))),
 ((7473734, (7.7714285714285705, 162.6)), (1669926, (48.269, 134.01))),
 ((7473734, (7.7714285714285705, 162.6)), (8141408, (-4.59, 137.44))),
 ((7473734, (7.7714285714285705, 162.6)), (8279514, (-4.59, 137.44))),
 ((7473734, (7.7714285714285705, 162.6)), (9946080, (-9.5, 54.4))),
 ((7473734, (7.7714285714285705, 162.6)), (347768, (41.05, 350.5)))]

In [ ]:
# (x[1][1][0]-x[0][1][0])**2+(x[1][1][1]-x[0][1][1])**2)**(1/2) es el calculo "a mano" de las distancias euclideas 
distance_between_ids = marsLocationById.cartesian(marsLocationById).map(lambda x: ((x[0][0],x[1][0]),((x[1][1][0]-x[0][1][0])**2+(x[1][1][1]-x[0][1][1])**2)**(1/2)))

In [ ]:
distance_between_ids.take(10)

[((7473734, 7473734), 0.0),
 ((7473734, 2963878), 10.155714135723322),
 ((7473734, 5773024), 49.57258709823651),
 ((7473734, 222604), 97.70954458005019),
 ((7473734, 559924), 49.57258709823651),
 ((7473734, 1669926), 49.57258709823651),
 ((7473734, 8141408), 28.032668733578156),
 ((7473734, 8279514), 28.032668733578156),
 ((7473734, 9946080), 109.569805352104),
 ((7473734, 347768), 190.82419478757544)]

Posible busqueda sin contar las distancias iguales a cero


In [ ]:
# shortest_distance = distance_between_ids.filter(lambda x: x[1] != 0).reduce(lambda x,y: x if x[1] < y[1] else y)

Posible busqueda contando las distancias iguales a cero pero de ids diferentes


In [ ]:
shortest_distance = distance_between_ids.filter(lambda x: x[0][0] != x[0][1]).reduce(lambda x,y: x if x[1] < y[1] else y)

In [ ]:
shortest_distance

((6105621, 8792365), 0.0)

In [ ]:
df_contents = sqlContext.read.parquet('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/contents.parquet', header=True, inferSchema=True)
rdd_contents = df_contents.rdd

In [ ]:
rdd_contents.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor_username='Lojwe

In [ ]:
closest_content_1 = rdd_contents.filter(lambda x: x.id == shortest_distance[0][0]).collect()[0][0]
closest_content_1

'Bradbury Landing'

In [ ]:
closest_content_2 = rdd_contents.filter(lambda x: x.id == shortest_distance[0][1]).collect()[0][0]
closest_content_2

'Rocknest (Marte)'

# Ejercicio 12
Obtenga la matriz de distancias euclídeas para todos los contenidos que están en Marte. ¿Cuáles son los dos contenidos que están a menor distancia?

# Respuesta


In [ ]:
print("Los dos contenidos más cercanos en marte son",closest_content_1,"y",closest_content_2,"con una distancia de",shortest_distance[1])

Los dos contenidos más cercanos en marte son Bradbury Landing y Rocknest (Marte) con una distancia de 0.0


Una muestra de la matriz euclidea, con formato ((ID<sub>1</sub>,ID<sub>2</sub>),distancia) es:


In [ ]:
distance_between_ids.take(10)

[((7473734, 7473734), 0.0),
 ((7473734, 2963878), 10.155714135723322),
 ((7473734, 5773024), 49.57258709823651),
 ((7473734, 222604), 97.70954458005019),
 ((7473734, 559924), 49.57258709823651),
 ((7473734, 1669926), 49.57258709823651),
 ((7473734, 8141408), 28.032668733578156),
 ((7473734, 8279514), 28.032668733578156),
 ((7473734, 9946080), 109.569805352104),
 ((7473734, 347768), 190.82419478757544)]